# Elder Scrolls Normal Map Generation via CycleGAN
Using implementation of CycleGAN here: https://github.com/LynnHo/CycleGAN-Tensorflow-2

The purpose of this document is to explain how to utilize CycleGANs to generate normal maps for input texture for The Elder Scrolls III: Morrowind's open-source engine reimplementation, OpenMW. 

This document is intended for advanced users. If you are not comfortable in the command line and in troubleshooting issues on your own, then this guide is not currently intended for you.

# Discussion of Network Architecture

First, let me be clear that I don't really understand how this network works under the hood. It's a tool for me. In the same way that I don't understand the details of how video cards work, or how microwaves work, I don't really know how this works. However, I am learning, and will continue to learn. 

If you do, please, add to this and enhance it! 

The purpose of this network is to generate Normal/Height maps from arbitrary input textures. To do this, we need to train the network on how to translate from an input texture to the desired normal/height map. We accomplish this with a CycleGAN. CycleGANs are a relatively new form of Deep Learning network that does not require paired data. We simply give it a selection of normals, and a selection of regular textures, and it learns how to translate back and forth between them. They are a fascinating subject, and there are some good resources around the internet explaining them.

The choice of input data is very important. We can't just take all of the normal/heightmaps in the broader Elder Scrolls modding world and dump them into a folder. (I did try that though.) We need to train the network on more specific groupings. 

My current attempt is to train this network with architecture - specifically, large relatively flat surfaces with mostly rectilinear components, such as tile floor or stone walls. I am using a selection Lysol's wonderful normalmaps (which you can find here: https://www.nexusmods.com/morrowind/users/34241390 ) with his permission to train this network. 

My first attempt ignored the alpha channel on normal maps, which proved to be a bad decision - a lot of data is encoded in that channel. This journal contains code the split the input normals into their component RGBA channels - Red, Green, Blue, and Alpha. It then resizes them to the necessary dimensions, moves them to the necessary locations, and returns a list of commands for you to run in the command prompt. 

After the networks have been trained on each of the channels, we take input textures and run them through each network. Finally, we combine the resulting single-channel normals into a single image, resize it to the original values, and - hopefully - we are done. 

# Step 0 - Preliminary Setup

First, get Tensorflow running on your computer. The official installation instructions are here: https://www.tensorflow.org/install/gpu. It is strongly recommended to use a GPU for this, which requires a modern Nvidia GPU. I am using an RTX 2060 non-super with 6GB VRAM - and I wish I had more.

Second, download the CycleGAN implementation listed above. Ensure that you have it working by training it on the test dataset mentioned in the repository. 

Place the folder containing the GAN in the folder with this notebook. I suggest creating a new folder for this purpose, as we will be creating many files and directories. If this is on your desktop or in your downloads folder, it will get messy.

Finally, install Imagemagick. https://imagemagick.org/index.php We will use this to mass-convert the finalized images to dds for easy use in Morrowind, Oblivion, or Skyrim.

# Step 1 - Dataset Preprocessing and Segmentation



The network operates on 256x256 square images. We need to adjust our images to fit this requirement. 

To accomplish this, I have written the following script that will take images in the subdirectory "Raw_Input" and convert them to the required size and format and store them in the "Processed_Input" directory.
    
You should have one set of folders each for Diffuse and Normals.

In the following blocks, we split the Heightmaps into four separate images - one for each channel R, G, B, and A. We will train four networks, one for each component, and then recombine the final images at the end. 

In [1]:
import os
import math
import time
import random
import shutil
from os import path
from PIL import Image
from PIL import ImageEnhance
from PIL import ImageOps

GAN_location = "./CycleGAN-Tensorflow-2-master/"

In [2]:
input_dirs = ['./Training/Raw_Input_Diffuse/','./Training/Raw_Input_Normal/']
target_dirs= ['./Processed_Input_Diffuse/', './Processed_Input_Normal/']
channels = ['R','G','B','A']

size = 256

# This method stores the original sizes of the images.
# We do not need this for the first pass, but we will use
# it later.

def prepare(target,channels):
    for i in range(len(channels)):
        if path.exists(target[1]+channels[i]):
            print("====== Using pre-existing resize directory ======")
        else:
            os.makedirs(target[1]+channels[i])
            
def convertAndResizeNormals(data_dir,target,channels):
    original_sizes = list()
        
    for item in os.listdir(data_dir):
        print("Processing: "+item)
        im=Image.open(data_dir+item)
        width,height = im.size
        original_sizes.append([item[0:-4],width,height])
        for i in range(4):    
            single_channel = im.split()[i-1]
            single_channel = single_channel.resize((size,size), Image.ANTIALIAS)
            single_channel = single_channel.convert("RGB")
            
            # Drop the last 4 characters, usually the previous filename ".dds"
            single_channel.save(target + channels[i-1] + '/' + item[0:-4] + ".jpg","JPEG")
        
        im.close()
    
    return original_sizes

def convertAndResizeDiffuse(data_dir,target):
    for item in os.listdir(data_dir):
        print("Processing: "+item)
        im=Image.open(data_dir+item)
        width,height = im.size
        im = im.resize((size,size), Image.ANTIALIAS)
        im = im.convert("RGB")
        # Drop the last 4 characters, usually the previous filename ".dds"
        im.save(target+item[0:-4]+".jpg","JPEG")
        
        im.close()
    
prepare(target_dirs,channels)
convertAndResizeDiffuse(input_dirs[0], target_dirs[0])
convertAndResizeNormals(input_dirs[1], target_dirs[1], channels)

====== Using pre-existing resize directory ======
====== Using pre-existing resize directory ======
====== Using pre-existing resize directory ======
====== Using pre-existing resize directory ======
Processing: 500wall.dds
Processing: cpstone01.dds
Processing: cpstone02.dds
Processing: cpstone03.dds
Processing: cpstone05.dds
Processing: cpstone06.dds
Processing: markarthtemp02.dds
Processing: mrkrockdesigns01.dds
Processing: mrkstoneblocks03.dds
Processing: mrktileslates01.dds
Processing: mrkwalkwaysa01.dds
Processing: skyhavenfloorstones01.dds
Processing: skyhavenstones01.dds
Processing: sovroof01.dds
Processing: sovstonestacked01.dds
Processing: tr_nec_floor.dds
Processing: tr_nec_floor.jpg
Processing: tr_nec_wall3_01.dds
Processing: tx_brickedge_left_01.dds
Processing: tx_brickedge_right_01.dds
Processing: tx_colony_floor03.jpg
Processing: tx_common_stone_floor.dds
Processing: tx_common_stone_floor.jpg
Processing: tx_dae_floor_01.jpg
Processing: tx_dwe_brick00.dds
Processing: tx_im

[['500wall_n', 4096, 4096],
 ['arenastonecolumn01bloody_n', 2048, 2048],
 ['arenastonecolumn01_n', 2048, 2048],
 ['arenastonegrey01_n', 2048, 2048],
 ['arenastonegrey02_n', 2048, 2048],
 ['arenastonegrey04_n', 2048, 2048],
 ['arenawoodwall01_n', 2048, 2048],
 ['cobblestonecheydinhal03_N', 512, 512],
 ['cpstone02_N', 512, 512],
 ['cpstone05_N', 512, 512],
 ['cpstone06_N', 512, 512],
 ['glasswindow10lowerclass_n', 256, 512],
 ['glasswindow10middleclass_n', 256, 512],
 ['glasswindow10_n', 256, 512],
 ['hhcol01_n', 4096, 4096],
 ['hhcol02_n', 4096, 4096],
 ['hhcol03_n', 4096, 4096],
 ['hhsteps01_n', 4096, 4096],
 ['markarthtemp02_n', 2048, 2048],
 ['mrkrockdesigns01_n', 2048, 2048],
 ['mrkstoneblocks03_n', 2048, 2048],
 ['mrktileslates01_n', 2048, 2048],
 ['mrkwalkwaysa01_n', 4096, 4096],
 ['riftenplazabrick01_n', 4096, 4096],
 ['riftenplazabrick02_n', 2048, 2048],
 ['riftenroofshingles01_n', 1024, 1024],
 ['riftenstonewall01_n', 2048, 2048],
 ['roof01_n', 1024, 1024],
 ['shipwoodside02_n'

Now we want to automatically segment the processed inputs into Test and Train data. Most of our images will be for Training. A few will be held back for testing. 

The following script will run on both of the Processed Input folders and put a random X% of the images into the testing set, and move the rest of them into the training set. X is 20 by default, but you can change that.

In [3]:
percent_to_move = 0.2
ahora = str(int(time.time()))

from_dirs=['./Processed_Input_Diffuse/', './Processed_Input_Normal/']

def generate_to_dirs(GAN_location, channels, ahora):
    to_dirs = list()
    for i in range(len(channels)):
        target = GAN_location+'datasets/'+channels[i]+'-'+ahora
        to_dirs.append(target)
        os.makedirs(target)
    return to_dirs

def find_number_of_files(directory):
    #print(directory)
    count = len([name for name in os.listdir(directory)])
    return count 

    
def segment_diffuse(from_dir,to_dirs,percent_to_move):
    count = find_number_of_files(from_dir)
    num_to_move = int(count * percent_to_move)
    #print(count)
    #print(num_to_move)
    #print ("Moving "+str(num_to_move)+" files to testing directory for"+from_dir)

    # For each color channel, randomly grab images to be test images, then assign
    # the rest as training images.
    for target in to_dirs:
        os.makedirs(target+'/TrainA')
        os.makedirs(target+'/TestA')
        # Grab the random files and move them to the Test Directory
        for i in range(num_to_move):
            count = find_number_of_files(from_dir)
            items = os.listdir(from_dir)
            to_move = items[random.randint(0,count-1)]
            #print("Moving file "+to_move)
            shutil.copy(from_dir + to_move, target+ '/TestA/' + to_move)
        items = os.listdir(from_dir)
        for item in items:
            shutil.copy(from_dir + item, target+ '/TrainA/' +item)
        


def segment_normals(from_dir,channels,percent_to_move):
    # For each color that we have, we want to move a random selection of files 
    # to the test directory, and then the rest to the train directory.
    for color in channels:
        # Figure out where we're putting the files
        target = GAN_location + 'datasets/' + color + '-' + ahora + '/TestB/'
        
        # Make the director that we're going to put stuff in
        os.makedirs(target)
        
        # Figure out exactly how many files we are going to move.
        count = find_number_of_files(from_dir + color)
        to_move = int(count * percent_to_move)
        
        # Now we enter the loop to move all of the files up to the percent specified
        for i in range(to_move):
            # Find how many files we're working with.
            count = find_number_of_files(from_dir + color)
            print(count)
            # Get the list of files.
            items = os.listdir(from_dir + color)
            # Select a random file to move to the testing directory
            to_move = items[random.randint(0,count-1)]
            print ("Moving file " + from_dir + color +'/' + to_move)
            shutil.copy(from_dir + color +'/' + to_move, target + to_move)
        
        # Finally, we move the remaining images to the training directory.
        items = os.listdir(from_dir + color)
        target = GAN_location + 'datasets/' + color + '-' + ahora + '/TrainB/'
        os.makedirs(target)

        for item in items:
            shutil.copy(from_dir + color + '/' + item, target + item)
    

to_dirs = generate_to_dirs(GAN_location, channels, ahora)
print(to_dirs)
segment_diffuse(from_dirs[0],to_dirs,percent_to_move)
print(channels)
segment_normals(from_dirs[1], channels, percent_to_move)

print( "====== Segmentation Complete ======" )
# TO DO: Make sure we don't train and test on same files. Make sure we don't copy same files.

['./CycleGAN-Tensorflow-2-master/datasets/R-1587222131', './CycleGAN-Tensorflow-2-master/datasets/G-1587222131', './CycleGAN-Tensorflow-2-master/datasets/B-1587222131', './CycleGAN-Tensorflow-2-master/datasets/A-1587222131']
['R', 'G', 'B', 'A']
78
Moving file ./Processed_Input_Normal/R/tx_imp_wall_02_nh.jpg
78
Moving file ./Processed_Input_Normal/R/tx_wood_ceilingpanel_01_nh.jpg
78
Moving file ./Processed_Input_Normal/R/cpstone02_N.jpg
78
Moving file ./Processed_Input_Normal/R/arenastonegrey04_n.jpg
78
Moving file ./Processed_Input_Normal/R/tx_imp_half_01_nh.jpg
78
Moving file ./Processed_Input_Normal/R/glasswindow10lowerclass_n.jpg
78
Moving file ./Processed_Input_Normal/R/hhcol03_n.jpg
78
Moving file ./Processed_Input_Normal/R/tx_bc_muck_nh.jpg
78
Moving file ./Processed_Input_Normal/R/hhsteps01_n.jpg
78
Moving file ./Processed_Input_Normal/R/tx_wall_brick_imp_01_nh.jpg
78
Moving file ./Processed_Input_Normal/R/arenastonegrey02_n.jpg
78
Moving file ./Processed_Input_Normal/R/tx_v_ba

# Step Two: Network Training

You now have prepared data, and can begin training your network. You will set the name of this folder below. This process takes a while.

Open a new CMD prompt and CD to the root of the GAN directory - the folder with "train.py" and the "datasets" folder in it, and run the following command.

This process will take a while, but you get a pretty chart to watch while it runs. We can also visualize the results every 100 generations from the Output folder.

In [4]:
# Get the command you need to run in the terminal
for color in channels:
    print('python train.py --dataset ' + color + '-' + ahora)

python train.py --dataset R-1587222131
python train.py --dataset G-1587222131
python train.py --dataset B-1587222131
python train.py --dataset A-1587222131


It is also possible to view the status of the network via Tensorboard, live. This can give you insight into why your network is performing the way it is. It's also cool looking. Run the following command from within the CycleGAN directory, and point your browser at http://localhost:6006/

In [5]:

# Get the command you need to run in the terminal
for color in channels:
    print('tensorboard --logdir output/' + color + '-' + ahora + ' --port 6006')

tensorboard --logdir output/R-1587222131 --port 6006
tensorboard --logdir output/G-1587222131 --port 6006
tensorboard --logdir output/B-1587222131 --port 6006
tensorboard --logdir output/A-1587222131 --port 6006


# Step Three: Normal Generation
The next step is to supply the model with input textures and get the normals that you want. This will involve some changes in the underlying code of the GAN, which I will explain later in this notebook.

Put the image that you want to generate normals for into the NormalGen/Input folder.

We will also have to scale the images back to the original resolution of the input, which requires storing what the resolution is for each input image. That is accomplished in the following block.

In [ ]:
import csv

targets = ['./NormalGen/Input/', './NormalGen/Output/']
input_folder = targets[0]
output_folder = targets[1]

def prepare():
    for item in targets:
        if path.exists(item):
            print (item + "directory Exists.")
        else:
            os.makedirs(item)
            print("Created " + item + " directory.")

prepare()

def convertAndResizeTargets(input_folder,output_folder):
    original_sizes = list()
    # Find all images that we are attempting to generate normals for.
    targets = os.listdir(input_folder)
    # Now, convert and resize all of them, and put them in the "Output" folder.
    for i in range(len(targets)):
        im = Image.open(input_folder + targets[i])
        width,height = im.size
        original_sizes.append([targets[i][0:-4],width,height])
        im=im.resize((size,size), Image.ANTIALIAS)
        im = im.convert("RGB")
        im.save(output_folder + targets[i][0:-4]+".jpg","JPEG")
    return original_sizes

original_sizes = convertAndResizeTargets(input_folder,output_folder)
print(original_sizes)

Next, we take these processed images and move them to the correct folder, and run the network on them. Since we just trained four networks, we need four copies of each file. 

We will generate four outputs - one each for each RGBA channel. 


In [ ]:
for item in os.listdir(output_folder):
    for color in channels:
        shutil.copy(output_folder + item, GAN_location+'datasets/'+ color + '-' + ahora + '/TestA/' + item)
        print("Moving " + item + ".")

Finally, we activate the network with the Train.py that I included in the main folder. It is a very slightly altered version of the "test.py" present in the original. The only change is to ensure that it only outputs the single image, rather than the compilation it does by default. Replace the original with the copy supplied. 

Again, we cannot directly run it. Use the following command from within the GAN folder. 

In [ ]:
for color in channels:
    print("python test.py --experiment_dir ./output/"+ color +'-' + ahora)

After this runs, we need to recombine and rescale the output images to match what they were originally. To do this, we will reference the original sizes that we stored earlier. 

In [ ]:
final_output_location = './Final_Output/'

           
# for each original image, find all of the components and recombine them.
# then rescale them.
for entry in original_sizes: 
    # find all of the original images
    # Initialize an empty list which we will store the individual channels in
    # prior to merging
    im = list()
    for color in channels:
        gan_output = GAN_location + 'output/' + color + '-' + ahora + '/samples_testing/A2B/'
        # We have to convert the image to a luminosity channel before merging.
        im.append(Image.open(gan_output + entry[0]+'.jpg').convert("L"))
    
    # The neural net currently generates alpha channels that are too bright
    # and that are lacking contrast. We attempt compensate for this
    # by lowering the brightness of all channels by half, and increasing the contrast
    # of the alpha channel.
    for i in range(len(im)):
        enhancer = ImageEnhance.Brightness(im[i])
        im[i] = enhancer.enhance(1)
    
    # I'm messing with the alpha layer here. At this point I'm just fucking around.
    im[3] = ImageOps.autocontrast(im[3])
    
    alpha_brightness = ImageEnhance.Brightness(im[3])
    im[3] = alpha_brightness.enhance(.5)
    # alpha_contrast = ImageEnhance.Contrast(im[3])
    # im[3] = alpha_contrast.enhance(2)
    
    
    # Now that we have all of the channels in a list, we merge
    # them back together
    print(im)
    new_img = Image.merge("RGBA",im)
    # Then we resize it to the original size.
    new_img = new_img.resize((entry[1],entry[2]), resample=Image.BICUBIC)
    # and finally, we save it as a PNG because it keeps transparency that way.
    new_img.save(final_output_location + entry[0] +'_nh'+'.png',"PNG")


print("Done!")

Finally, we need to convert all of the images to dds. This is accomplished via imagemagick. Go to the output folder and run the following:

> magick mogrtify -format dds *

Once that's done, go ahead and put them in a folder and add the folder path to your OpenMW cfg as a mod. 

Congratulations, you're done! 

# To Do: 

- Investigate splitting each image into R, G, B, and A components and creating separate GANs for each, then recombining at the end. The current method loses alpha channel information, which is very important for some of these height or normal maps. See: https://stackoverflow.com/questions/51325224/python-pil-image-split-to-rgb/51555134 for potential way to do this.

- Also here for plotting: https://stackoverflow.com/questions/55677216/how-to-convert-an-rgba-image-to-grayscale-in-python

- See here for a better way to get the Alpha https://stackoverflow.com/questions/1962795/how-to-get-alpha-value-of-a-png-image-with-pil

- Try training another GAN to go from the Alphas that I generate in the first pass, to better Alphas. Perhaps they can be improved...

- Find additional training data.

# Splitting into each color channel

Preliminary investigations are showing me that the transfer from RGBA space to RGB space is, to put it bluntly, awful. It removes a ton of the detail that make textures like Lysol's so nice, and removes an entire dimension from the possible training space. 

I would like to try out splitting the normals into multiple channels, then training models to generate one channel each. 

This would involve a refactor of the code a bit. I would need to first split each image, then resize and move around, then create four "dataset" dirs, one for each channel. I will use the same input images. 



Lowering Cycle Loss Weight Over Time

https://ssnl.github.io/better_cycles/report.pdf See that. Basically, the image is required to be translatable back and forth, which forces the gan to require encoding the necessary information within the image in a way that minimally disrupts the cycle loss weight, but which is still available for it to get back to the original image. It develops its own encoding each time. 

Keeping the images reversible is a good idea, since it constraints the possible weights, but it can also enforce this lossless encoding. We don't always want it to be lossless. So instead, let's try lowering the cycle loss weight over time. Right now, cycle loss weight has a default value of 10. What if we lower that over time? 

I'm going to try implementing this. Adjusting Train.py. I have set it so that the cycle loss weight will go scale down linearly throughout the training process, with an end-state of half of the starting state.

Training on a large image corpus

I'm interesting in seeing what happens if I train it on an much larger group of images. Most GANs require an incredibly large corpus to perform well. So far, I've been training on the order of tens of images. I want to scale that up to thousands. 

To test this, I think that I will simply grab _every_ normal that I have, from Morrowind, Oblivion, and Skyrim, and have at it. I will also grab a TON of regular texture files. 

Testing on larger images. Can my GPU handle them? - NOPE! Tried on 512x512, got out of memory errors. Now trying 300x300, because why not? Looks like it's working.

In [ ]:
Testing on larger batches. Can my GPU handle them?